In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Parse all xml files to numeric format**

In [0]:
from bs4 import BeautifulSoup
import os
import csv

folder = '/content/drive/My Drive/KTH_Handtool_Dataset'
subfolder = ['Blue_background']
in_subfolder = ['Artificial', 'Cloudy', 'Directed']
instruments = ['hammer1', 'hammer2', 'hammer3', 'plier1', 'plier2', 'plier3', 'screw1', 'screw2', 'screw3']
sub_instruments = ['Kinect', 'webcam']

with open('employee_file.csv', mode='w') as file:
  for folder_name in subfolder:
    for in_folder_name in in_subfolder:
      for instrument in instruments:
        directory_name = os.path.join(folder, folder_name, 'rgb', in_folder_name, instrument)
        directory_name_xml = os.path.join(folder, folder_name, 'bboxes', in_folder_name, instrument)
        for filename in os.listdir(directory_name_xml):
          label = instrument
          filename_jpg = filename[:-4] + '.jpg'
          filename_str = os.path.join(directory_name, filename_jpg)
          handler = open(os.path.join(directory_name_xml, filename)).read()
          soup = BeautifulSoup(handler, "xml")
          xmin = int(soup.xmin.string)
          xmax = int(soup.xmax.string)
          ymin = int(soup.ymin.string)
          ymax = int(soup.ymax.string)
          row = [filename_str, xmin, ymin, xmax, ymax, label]
          writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
          writer.writerow(row)
csvfile.close()

In [0]:
/content/drive/My Drive/KTH_Handtool_Dataset/Blue_background/bboxes/Artificial/hammer1/camera1_15.xml

In [0]:
subfolder = ['Brown_background', 'White_background']